# LiteWaste training notebook

In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

## constants

In [ ]:
DATA_PATH = "./data/"
BATCH_SIZE = 32
IMAGE_RESOLUTION = (224, 224)
EPOCHS = 40
CLASS_COUNT = len(os.listdir(dataset_dir))

## data preparation

In [ ]:
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.125,
    height_shift_range=0.125,
    shear_range=0.25,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
    brightness_range=[0.75, 1.25],
    channel_shift_range=16,
)

augmented_images = []
for i in range(4):
    augmented_images.extend(datagen.flow_from_directory(
            DATA_PATH,
            target_size=IMAGE_RESOLUTION,
            batch_size=BATCH_SIZE,
            class_mode="categorical",
    ))

## model definition

In [ ]:
base_model = VGG19(weights="imagenet", include_top=False)
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation="relu")(x)
predictions = Dense(num_classes, activation="softmax")(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

## compilation

In [ ]:
model.compile(optimizer=Adam(learning_rate=0.0001),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

## training

In [ ]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // BATCH_SIZE,
    epochs=EPOCHS,
)